In [2]:
import sys
sys.path.append('..')
wp = '/home/jose/lung_cancer_ds_bowl/'

In [6]:
import os
import random
import logging
import multiprocessing
import numpy as np
import pandas as pd
from time import time
import matplotlib.pyplot as plt
from utils import plotting
from dl_model_pos_patches import  common
from sklearn import metrics
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback
from dl_networks.sample_resnet import ResnetBuilder
from dl_utils.tb_callback import TensorBoard

In [8]:
INPUT_PATH = '/mnt/hd2/preprocessed5'
VALIDATION_PATH = '/mnt/hd2/preprocessed5_validation_luna'
NODULES_PATH = wp + 'data/luna/annotations.csv'
PATCHES_PATH = '/mnt/hd2/patches'

K.set_image_dim_ordering('th')

In [13]:
### PATCHES GENERATION -----------------------------------------------------------------

## PATIENTS FILE LIST
patients_with_annotations = pd.read_csv(NODULES_PATH)  # filter patients with no annotations to avoid having to read them
patients_with_annotations = list(set(patients_with_annotations['seriesuid']))
patients_with_annotations = ["luna_%s.npz" % p.split('.')[-1] for p in patients_with_annotations]

filenames = os.listdir(INPUT_PATH)
filenames = [g for g in filenames if g.startswith('luna_')]
filenames_train = [os.path.join(INPUT_PATH, fp) for fp in filenames if fp in patients_with_annotations]
filenames_test = [os.path.join(VALIDATION_PATH, fp) for fp in os.listdir(VALIDATION_PATH) if fp in patients_with_annotations]
filenames_dsb = [os.path.join(INPUT_PATH, g) for g in filenames if g.startswith('dsb_')]

In [11]:
def __load_and_store(filename):
    patient_data = np.load(filename)['arr_0']
    X, y, rois, stats = common.load_patient_3d_atlas(patient_data, 
                                                     discard_empty_nodules=True, output_rois=True, debug=True, thickness=1
                                                    )
    #logging.info("Patient: %s, stats: %s" % (filename.split('/')[-1], stats))
    return X, y, stats

In [ ]:
for filepath in filenames_train[0]:
    common.multiproc_crop_generator(filenames_train,
                                os.path.join(PATCHES_PATH,(filepath.split('/')[-1]+'_x.npz')),
                                os.path.join(PATCHES_PATH,(filepath.split('/')[-1]+'_y.npz')),
                                __load_and_store, store=True)

04-05 17:54:54  INFO     Loading 0/583
04-05 17:54:58  INFO     ++ Slice 104, stats: {'fp': 25, 'tp': 1, 'fn': 0}
04-05 17:54:58  INFO     ++ Slice 105, stats: {'fp': 24, 'tp': 1, 'fn': 0}
04-05 17:54:58  INFO     ++ Slice 106, stats: {'fp': 27, 'tp': 1, 'fn': 0}
04-05 17:54:58  INFO     ++ Slice 107, stats: {'fp': 22, 'tp': 1, 'fn': 0}
04-05 17:54:58  INFO     ++ Slice 108, stats: {'fp': 30, 'tp': 1, 'fn': 0}
04-05 17:54:58  INFO     Loading 1/583


(17, 20, 20)


04-05 17:55:02  INFO     ++ Slice 98, stats: {'fp': 31, 'tp': 1, 'fn': 0}
04-05 17:55:02  INFO     ++ Slice 99, stats: {'fp': 29, 'tp': 1, 'fn': 0}
04-05 17:55:02  INFO     ++ Slice 100, stats: {'fp': 33, 'tp': 1, 'fn': 0}
04-05 17:55:02  INFO     ++ Slice 138, stats: {'fp': 25, 'tp': 1, 'fn': 0}
04-05 17:55:02  INFO     ++ Slice 139, stats: {'fp': 25, 'tp': 1, 'fn': 0}
04-05 17:55:02  INFO     ++ Slice 140, stats: {'fp': 26, 'tp': 1, 'fn': 0}
04-05 17:55:02  INFO     Loading 2/583


(17, 20, 20)


04-05 17:55:06  INFO     ++ Slice 66, stats: {'fp': 64, 'tp': 1, 'fn': 0}
04-05 17:55:06  INFO     ++ Slice 67, stats: {'fp': 61, 'tp': 1, 'fn': 0}
04-05 17:55:06  INFO     ++ Slice 68, stats: {'fp': 55, 'tp': 1, 'fn': 0}
04-05 17:55:06  INFO     ++ Slice 85, stats: {'fp': 35, 'tp': 1, 'fn': 0}
04-05 17:55:06  INFO     Loading 3/583


...

(17, 20, 20)


04-05 17:56:59  INFO     ++ Slice 85, stats: {'fp': 30, 'tp': 1, 'fn': 0}
04-05 17:56:59  INFO     ++ Slice 86, stats: {'fp': 27, 'tp': 1, 'fn': 0}
04-05 17:56:59  INFO     ++ Slice 87, stats: {'fp': 35, 'tp': 1, 'fn': 1}
04-05 17:56:59  INFO     ++ Slice 88, stats: {'fp': 31, 'tp': 1, 'fn': 0}
04-05 17:56:59  INFO     ++ Slice 89, stats: {'fp': 28, 'tp': 1, 'fn': 0}
04-05 17:56:59  INFO     ++ Slice 100, stats: {'fp': 39, 'tp': 1, 'fn': 0}
04-05 17:56:59  INFO     ++ Slice 101, stats: {'fp': 44, 'tp': 1, 'fn': 0}
04-05 17:56:59  INFO     ++ Slice 102, stats: {'fp': 34, 'tp': 1, 'fn': 0}
04-05 17:57:00  INFO     ++ Slice 103, stats: {'fp': 36, 'tp': 1, 'fn': 0}
04-05 17:57:00  INFO     Loading 38/583


(16, 20, 20)


04-05 17:57:01  INFO     ++ Slice 122, stats: {'fp': 58, 'tp': 1, 'fn': 0}
04-05 17:57:01  INFO     ++ Slice 123, stats: {'fp': 60, 'tp': 1, 'fn': 0}
04-05 17:57:01  INFO     ++ Slice 124, stats: {'fp': 59, 'tp': 1, 'fn': 0}
04-05 17:57:01  INFO     ++ Slice 125, stats: {'fp': 58, 'tp': 1, 'fn': 0}
04-05 17:57:02  INFO     ++ Slice 126, stats: {'fp': 59, 'tp': 1, 'fn': 0}
04-05 17:57:02  INFO     ++ Slice 127, stats: {'fp': 69, 'tp': 1, 'fn': 0}
04-05 17:57:02  INFO     ++ Slice 128, stats: {'fp': 68, 'tp': 1, 'fn': 0}
04-05 17:57:02  INFO     ++ Slice 129, stats: {'fp': 65, 'tp': 1, 'fn': 0}
04-05 17:57:02  INFO     ++ Slice 130, stats: {'fp': 60, 'tp': 1, 'fn': 0}
04-05 17:57:02  INFO     Loading 39/583


(16, 20, 20)
